In [0]:
# imports

import os

In [0]:
# Creating a set of classes to simplify LLM use

from abc import ABC, abstractmethod
from dotenv import load_dotenv
# Imports for type definition
from collections.abc import MutableSequence
from typing import TypedDict

class LLM_Wrapper(ABC):
    """
    The parent (abstract) class to specific LLM classes, normalising and providing common 
    and simplified ways to call LLMs while adding some level of abstraction on
    specifics
    """

    MessageEntry = TypedDict('MessageEntry', {'role': str, 'content': str})
    
    system_prompt: str                   # The system prompt used for the LLM
    user_prompt: str                     # The user prompt
    __api_key: str                       # The (private) api key
    temperature: float = 0.5             # Default temperature
    __msg: MutableSequence[MessageEntry] # Message builder

    def __init__(self, system_prompt:str, user_prompt:str, env_apikey_var:str=None):
        """
        env_apikey_var: str # The name of the env variable where to find the api_key
                            # We store the retrieved api_key for future calls
        """
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        if env_apikey_var:
            load_dotenv(override=True)
            self.__api_key = os.getenv(env_apikey_var)

        # # API Key format check
        # if env_apikey_var and self.__api_key:
        #     print(f"API Key exists and begins {self.__api_key[:8]}")
        # else:
        #     print("API Key not set")
            
    def setSystemPrompt(self, prompt:str):
        self.system_prompt = prompt

    def setUserPrompt(self, prompt:str):
        self.user_prompt = prompt

    def setTemperature(self, temp:float):
        self.temperature = temp

    def getKey(self) -> str:
        return self.__api_key

    def messageSet(self, message: MutableSequence[MessageEntry]):
        self.__msg = message

    def messageAppend(self, role: str, content: str):
        self.__msg.append(
            {"role": role, "content": content}
        )

    def messageGet(self) -> MutableSequence[MessageEntry]:
        return self.__msg
        
    @abstractmethod
    def getResult(self):
        pass



In [0]:
from openai import OpenAI

class GPT_Wrapper(LLM_Wrapper):

    MODEL:str = 'gpt-4o-mini'
    llm:OpenAI

    def __init__(self, system_prompt:str, user_prompt:str):
        super().__init__(system_prompt, user_prompt, "OPENAI_API_KEY")
        self.llm = OpenAI()
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])


    def setSystemPrompt(self, prompt:str):
        super().setSystemPrompt(prompt)
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])

    def setUserPrompt(self, prompt:str):
        super().setUserPrompt(prompt)
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])

    def getResult(self, format=None):
        """
        format is sent as an adittional parameter {"type", format}
        e.g. json_object
        """
        if format:
            response = self.llm.chat.completions.create(
                model=self.MODEL,
                messages=super().messageGet(),
                temperature=self.temperature,
                response_format={"type": "json_object"}
            )
            if format == "json_object":
                result = json.loads(response.choices[0].message.content)
            else:
                result = response.choices[0].message.content
        else:
            response = self.llm.chat.completions.create(
                model=self.MODEL,
                messages=super().messageGet(),
                temperature=self.temperature
            )
            result = response.choices[0].message.content
        return result

In [0]:
import ollama

class Ollama_Wrapper(LLM_Wrapper):

    MODEL:str = 'llama3.2'

    def __init__(self, system_prompt:str, user_prompt:str):
        super().__init__(system_prompt, user_prompt, None)
        self.llm=ollama
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])


    def setSystemPrompt(self, prompt:str):
        super().setSystemPrompt(prompt)
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])

    def setUserPrompt(self, prompt:str):
        super().setUserPrompt(prompt)
        super().messageSet([
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt}
        ])

    def getResult(self, format=None):
        """
        format is sent as an adittional parameter {"type", format}
        e.g. json_object
        """
        response = self.llm.chat(
            model=self.MODEL, 
            messages=super().messageGet()
        )
        result = response['message']['content']
        return result

In [0]:
import anthropic

class Claude_Wrapper(LLM_Wrapper):

    MODEL:str = 'claude-3-5-haiku-20241022'
    MAX_TOKENS:int = 200
    llm:anthropic.Anthropic

    def __init__(self, system_prompt:str, user_prompt:str):
        super().__init__(system_prompt, user_prompt, "ANTHROPIC_API_KEY")
        self.llm = anthropic.Anthropic()
        super().messageSet([
            {"role": "user", "content": self.user_prompt}
        ])

    def setSystemPrompt(self, prompt:str):
        super().setSystemPrompt(prompt)

    def setUserPrompt(self, prompt:str):
        super().setUserPrompt(prompt)
        super().messageSet([
            {"role": "user", "content": self.user_prompt}
        ])

    def getResult(self, format=None):
        """
        format is sent as an adittional parameter {"type", format}
        e.g. json_object
        """
        response = self.llm.messages.create(
            model=self.MODEL,
            max_tokens=self.MAX_TOKENS,
            temperature=self.temperature,
            system=self.system_prompt,
            messages=super().messageGet()
        )
        result = response.content[0].text
        return result

In [0]:
import google.generativeai

class Gemini_Wrapper(LLM_Wrapper):

    MODEL:str = 'gemini-1.5-flash'
    llm:google.generativeai.GenerativeModel

    def __init__(self, system_prompt:str, user_prompt:str):
        super().__init__(system_prompt, user_prompt, "GOOGLE_API_KEY")
        self.llm = google.generativeai.GenerativeModel(
            model_name=self.MODEL,
            system_instruction=self.system_prompt
        )
        google.generativeai.configure(api_key=super().getKey())

    def setSystemPrompt(self, prompt:str):
        super().setSystemPrompt(prompt)

    def setUserPrompt(self, prompt:str):
        super().setUserPrompt(prompt)

    def getResult(self, format=None):
        """
        format is sent as an adittional parameter {"type", format}
        e.g. json_object
        """
        response = self.llm.generate_content(self.user_prompt)
        result = response.text
        return result